In [14]:
# import libraries
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd 
import plotly.express as px

In [15]:
df = pd.read_csv('Data/2018-2020_df.csv')

In [16]:
df.head(5)

,year,locationabbr,locationdesc,geographiclevel,datasource,class,topic,data_value,data_value_unit,data_value_type,stratificationcategory1,stratification1,stratificationcategory2,stratification2,topicid,locationid,y_lat,x_lon,data_value_footnote_symbol,data_value_footnote
0,2019,AK,Aleutians East,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,182.4,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Gender,Overall,Race/Ethnicity,Overall,T2,2013,55.442393,-161.959930,NaN,NaN
1,2019,AK,Aleutians West,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,172.6,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Gender,Overall,Race/Ethnicity,Overall,T2,2016,53.648343,-166.917533,NaN,NaN
2,2019,AK,Anchorage,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,255.6,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Gender,Overall,Race/Ethnicity,Overall,T2,2020,61.159145,-149.104559,NaN,NaN
3,2019,AK,Bethel,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,343.4,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Gender,Overall,Race/Ethnicity,Overall,T2,2050,60.923648,-159.753262,NaN,NaN
4,2019,AK,Bristol Bay,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,NaN,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Gender,Overall,Race/Ethnicity,Overall,T2,2060,58.753496,-156.694662,~,Insufficient Data


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59094 entries, 0 to 59093
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   year                        59094 non-null  int64  
 1   locationabbr                59094 non-null  object 
 2   locationdesc                59094 non-null  object 
 3   geographiclevel             59094 non-null  object 
 4   datasource                  59094 non-null  object 
 5   class                       59094 non-null  object 
 6   topic                       59094 non-null  object 
 7   data_value                  33087 non-null  float64
 8   data_value_unit             59094 non-null  object 
 9   data_value_type             59094 non-null  object 
 10  stratificationcategory1     59094 non-null  object 
 11  stratification1             59094 non-null  object 
 12  stratificationcategory2     59094 non-null  object 
 13  stratification2             590

In [18]:
# check missing values
df.isnull().sum()

year                              0
locationabbr                      0
locationdesc                      0
geographiclevel                   0
datasource                        0
class                             0
topic                             0
data_value                    26007
data_value_unit                   0
data_value_type                   0
stratificationcategory1           0
stratification1                   0
stratificationcategory2           0
stratification2                   0
topicid                           0
locationid                        0
y_lat                            18
x_lon                            18
data_value_footnote_symbol    33087
data_value_footnote           33087
dtype: int64

In [19]:
df.dropna(subset=['y_lat', 'x_lon'], inplace=True)
df = df.drop(columns=['geographiclevel', 'datasource', 'class', 'topic', 'data_value_footnote_symbol', 'data_value_footnote', 'stratificationcategory1', 'stratificationcategory2','topicid', 'data_value_type', 'data_value_unit'])
df.head(5)

,year,locationabbr,locationdesc,data_value,stratification1,stratification2,locationid,y_lat,x_lon
0,2019,AK,Aleutians East,182.4,Overall,Overall,2013,55.442393,-161.959930
1,2019,AK,Aleutians West,172.6,Overall,Overall,2016,53.648343,-166.917533
2,2019,AK,Anchorage,255.6,Overall,Overall,2020,61.159145,-149.104559
3,2019,AK,Bethel,343.4,Overall,Overall,2050,60.923648,-159.753262
4,2019,AK,Bristol Bay,NaN,Overall,Overall,2060,58.753496,-156.694662


In [20]:
column_names = {'stratification1': 'Gender', 'stratification2': 'race/ethnicityage_group'}
df = df.rename(columns=column_names)
df.reset_index(drop=True, inplace=True)
df.head(5)

,year,locationabbr,locationdesc,data_value,Gender,race/ethnicityage_group,locationid,y_lat,x_lon
0,2019,AK,Aleutians East,182.4,Overall,Overall,2013,55.442393,-161.959930
1,2019,AK,Aleutians West,172.6,Overall,Overall,2016,53.648343,-166.917533
2,2019,AK,Anchorage,255.6,Overall,Overall,2020,61.159145,-149.104559
3,2019,AK,Bethel,343.4,Overall,Overall,2050,60.923648,-159.753262
4,2019,AK,Bristol Bay,NaN,Overall,Overall,2060,58.753496,-156.694662
...,...,...,...,...,...,...,...,...,...
59089,2019,WY,Wyoming,NaN,Male,Asian and Pacific Islander,56,42.998400,-107.552000
59090,2019,WY,Wyoming,NaN,Female,Asian and Pacific Islander,56,42.998400,-107.552000
59091,2019,WY,Wyoming,338.4,Overall,American Indian and Alaskan Native,56,42.998400,-107.552000
59092,2019,WY,Wyoming,432.2,Male,American Indian and Alaskan Native,56,42.998400,-107.552000


In [23]:
df.to_csv('Data/2018-2020_complete_df.csv', index=False)